In [1]:
# 3.3.1生成数据集
from mxnet import autograd,nd

num_inputs= 2
num_examples= 1000
true_w = [2,-3.4]
true_b = 4.2
features = nd.random.normal(scale=1,shape=(num_examples,num_inputs))
#features[:,0]截取所有行，0列的元素，获得一个一维行向量,即1x1000
labels = true_w[0] *features[:,0]+true_w[1] * features[:,1]+true_b 
labels +=nd.random.normal(scale=0.01,shape=labels.shape)
features.shape,labels.shape# features为1000x2,labels为1x1000

((1000, 2), (1000,))

In [2]:
# 3.3.2 读取数据 
from mxnet.gluon import data as gdata

batch_size = 10
# 将训练数据的特征和标签组合
dataset = gdata.ArrayDataset(features,labels)
# 随机读取batc_size大小的数据样本，shuffle代表随机,data_iter为一个迭代器
data_iter =gdata.DataLoader(dataset,batch_size,shuffle=True)

In [3]:
for X,y in data_iter:
    print(X,y)
    break


[[-0.7265826   1.1108662 ]
 [ 0.70998806  0.03483093]
 [-1.2080863   1.8140212 ]
 [-0.19679789 -0.13781768]
 [-0.3216947  -0.83062977]
 [ 0.277957    1.1219468 ]
 [ 0.80678344 -0.5540094 ]
 [ 0.7201801   0.9385755 ]
 [ 0.23789974  0.41007295]
 [-0.0446528   1.1711494 ]]
<NDArray 10x2 @cpu(0)> 
[-1.0402609   5.4816027  -4.3862033   4.2766104   6.375893    0.9473662
  7.703524    2.4503956   3.2905803   0.12103113]
<NDArray 10 @cpu(0)>


In [4]:
# 3.3.3 定义模型
# nn是netural network的缩写，nn模块中定义了大量神经网路的层
from mxnet.gluon import nn
# 模型变量net，它是⼀个Sequential实例。在Gluon中，Sequential实例可以看作是⼀个串联各个层的容器。
# 在构造模型时，我们在该容器中依次添加层。
net = nn.Sequential()
# 线性回归的输出层⼜叫全连接层。
# 在Gluon中，全连接层是⼀个Dense实例。我们定义该层输出个数为1。 
# add函数向模型变量net中添加全连接层
net.add(nn.Dense(1))
# 在Gluon中我们⽆须指定每⼀层输⼊的形状，例如线性回归的输⼊个数。
# 当模型得到数据时，例如后⾯执⾏net(X)时，模型将⾃动推断出每⼀层的输⼊个数。

sequential0_ (

)
sequential0_ (
  Parameter dense0_weight (shape=(1, 0), dtype=float32)
  Parameter dense0_bias (shape=(1,), dtype=float32)
)


In [5]:
# 3.3.4 初始化模型参数 
# init是initializer的缩写形式,init模块提供了模型参数初始化的各种⽅法
from mxnet import init
# init.Normal(sigma=0.01)指定权重参数w每个元素将在初始化时随机采样于均值为0、标准差为0.01的正态分布。
# 偏差参数b默认会初始化为零。 net中有模型参数w，b
net.initialize(init.Normal(sigma=0.01))

In [6]:
# 3.3.5 定义损失函数 
# 在Gluon中，loss模块定义了各种损失函数。
from mxnet.gluon import loss as gloss
loss =gloss.L2Loss() # 平方损失又称L2范数损失 

In [7]:
# 3.3.6 定义优化算法
# 在导⼊Gluon后，我们创建⼀个Trainer实例，并指定学习率为0.03的小批量随机梯度下降（sgd）为优化算法。
from mxnet import gluon
# net.collect_params()可以获取net实例所有层的全部参数，sgd算法将迭代这些参数,学习率为0.03
trainer =gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.03})

In [8]:
# 3.3.7训练模型
num_epochs =3 
for epoch in range(1,num_epochs+1):
    for X,y in data_iter:
        with autograd.record():
            l = loss(net(X),y)# 小批量样本的loss，net(X)为模型预测值，y为真实值
        l.backward()# 对l求模型参数w,b的梯度
        # 在step函数中指明批量⼤小，从而对批量中样本梯度求平均
        trainer.step(batch_size)# 模型参数用sgd算法进行迭代
    l=loss(net(features),labels)# 所有样本的loss
    print('epoch %d,loss %f' %(epoch,l.mean().asnumpy()))

epoch 1,loss 0.035142
epoch 2,loss 0.000126
epoch 3,loss 0.000048


In [9]:
dense = net[0]
true_w,dense.weight.data()

([2, -3.4],
 
 [[ 1.9996562 -3.3999028]]
 <NDArray 1x2 @cpu(0)>)

In [10]:
true_b,dense.bias.data()

(4.2,
 
 [4.200024]
 <NDArray 1 @cpu(0)>)

In [11]:
dense.weight.grad(),dense.bias.grad()

(
 [[0.02962869 0.02212788]]
 <NDArray 1x2 @cpu(0)>,
 
 [0.0259296]
 <NDArray 1 @cpu(0)>)

In [13]:
help(dense.weight)

Help on Parameter in module mxnet.gluon.parameter object:

class Parameter(builtins.object)
 |  A Container holding parameters (weights) of Blocks.
 |  
 |  :py:class:`Parameter` holds a copy of the parameter on each :py:class:`Context` after
 |  it is initialized with ``Parameter.initialize(...)``. If :py:attr:`grad_req` is
 |  not ``'null'``, it will also hold a gradient array on each :py:class:`Context`::
 |  
 |      ctx = mx.gpu(0)
 |      x = mx.nd.zeros((16, 100), ctx=ctx)
 |      w = mx.gluon.Parameter('fc_weight', shape=(64, 100), init=mx.init.Xavier())
 |      b = mx.gluon.Parameter('fc_bias', shape=(64,), init=mx.init.Zero())
 |      w.initialize(ctx=ctx)
 |      b.initialize(ctx=ctx)
 |      out = mx.nd.FullyConnected(x, w.data(ctx), b.data(ctx), num_hidden=64)
 |  
 |  Parameters
 |  ----------
 |  name : str
 |      Name of this parameter.
 |  grad_req : {'write', 'add', 'null'}, default 'write'
 |      Specifies how to update gradient to grad arrays.
 |  
 |      - ``'wr